# RAL "Statistical Analysis Lab" Prerequisite Material

Please get yourself familiar with the following terminology and basic `RooFit` functionality. If you get any strange behaviour (weird errors) with any of the exercises it might be worth restarting the kernel and reloading the cells up to that point (under 'Kernel' menu above select 'Restart Kernel and run up to selected cell'). 

In [ ]:
import roofitbasics as rs
import ROOT

## Basic Statistical Analysis Terminology

Here are some generic definitions of things involved with statistical analysis. Make sure you understand them abstractly before we go on to see how they are represented in `RooFit`:

| Term | Description |
|------|-------------|
| Variable | A fundamental type that has a value. They are often a continuous real-valued quantity but can also be a discrete (categorical) valued quantity. |
| Observable | A variable that features in a dataset, representing a measurable quantity.  |
| Global Observable | A special type of observable in a dataset that takes a value that is independent of the events, so it has the same value for every event and is defined even if the dataset has no events in it. |
| Regular Observable | An observable that is not a global observable. |
| Dataset | A collection of events that each have a set of values for the regular observables of the dataset, and potentially also a weight. The dataset may also have a collection of global observable values. Think of the events like rows (entries) in a table and the regular observables are the columns. The global observables are an extra set of values that sit alongside the table. Examples of regular observables are channel category and bin number. An example of a global observable is the integrated luminosity. |
| PDF | A function of variables that, when evaluated, will return a probability density defined over a particular subset of the variables that is called the *normalization set* of the PDF. All the variables in the normalization set are considered to be observables. 
| Parameter | A variable of a function that is not an observable (i.e. for PDFs is a variable of the PDF that is not in the normalization set).
| Likelihood | A function of parameters defined by the combination of a PDF with a Dataset (the observables of the dataset define the normalization set of the PDF). The value of the likelihood function = $p(g)\prod_i p(o_{i})^{w_i}$ (where $g$ are the global observables, $o_i$ and $w_i$ are the observables and weight respectively of the ith entry of the dataset) and often the Negative Log Likelihood (NLL) is evaluated instead: -($\Sigma_i w_i log(p(o_i)) +log(p(g)))$


In RooFit (as well as most other statistical analysis packages) the actual value of the likelihood function may not correspond to precisely the likelihood, but may instead include a multiplicative factor (additive in NLL) that depends only on data (i.e. does not depend on the parameters). This is acceptable because it is never the absolute value of likelihood that we are interested in, but we study likelihood ratios and delta-log-likelihood, so this factor cancels out. 

Datasets can be binned or unbinned. When a dataset is binned this means all the events that fall in a given bin of some observable are combined into a single entry, with the observable value normally chosen to be the bin center, and the entry's weight equal to the sum (of weights) of the events in the bin. 


## Basic RooFit functionality

This notebook will cover only the essential RooFit concepts that you need to understand for the following statistical analysis tutorial. RooFit has a vast set of capabilities and functionality, and this notebook is careful to tread a particular path through the toolkit to expose you to only the essential concepts and **nothing more**. Please do not be afraid to stray from the path if you are interested in exploring further, but know that, sadly, what you find in other parts of RooFit do not always appear to be what they seem or behave how you might first think.

The headings of each section contain links to the ROOT documentation of those classes (note the documentation always refers to the C++ classes, which we are accessing via ROOT's python bindings).

### Variables: [RooRealVar](https://root.cern.ch/doc/v624/classRooRealVar.html) and [RooCategory](https://root.cern.ch/doc/v624/classRooCategory.html)
There are two fundamental variables types in `RooFit`:

  1. `RooRealVar` is a continuous variable, with an optional range defined by a lower and upper bound. They can also be optionally given an associated error. It can also have a binning, which can be relevent for observables, but this is not a feature we will make use of.
  2. `RooCategory`is a discrete variable with a defined set of possible values.
  
`RooRealVar` will appear as parameters of functions and as observables in datasets. `RooCategory` you will only encounter being used as an observable that identifies which *channel* an event lives in (the discrete values represent the different channels). Since we won't encounter them otherwise, we will focus on interacting with `RooRealVar`:

| Method | Description |
|--------|-------------|
|`v.GetName()`| Name of the variable |
|`v.isCategory()` | flag that tells if you have a `RooCategory` or a `RooRealVar` |
|`r.getVal()`| Access current value of a `RooRealVar`|
|`c.getIndex()`| Access current value of a `RooCategory`|
|`r.setConstant(True)`| Mark the var constant (use `False` to mark it floating) |
|`r.isConstant()` | Is the variable constant? (can also do `r.getAttribute("Constant")`) |
|`r.getMin()` | Get the lower boundary of the range (lowest possible value of variable) |
|`r.getMax()` | Get the upper boundary of the range (highest possible value of variable) |
|`r.setVal(x)`| change the value of the variable (respects range boundaries i.e. you cannot set value outside of the min and max) |
|`r.setRange(low,high)`| change the range of the RooRealVar |
|`r.hasError()`| does the var have a defined error? |
|`r.getError()`| get the error |
|`r.setError(x)`| set the error |
|`r.Print()`| shows current value, error (if defined), range, and constant status (marked by a "C" for constant) |


In [ ]:
r = rs.getObject("myVar") # gives you access to a variable ... 
# play with it below before answering the tests in the following cell

# play here with access r, e.g.
r.Print()

In [ ]:
## For all the exercises you put your answers inside the test functions (rs.test_XX(...))

rs.test_1a( True/False ) # is r constant? Delete as appropriate

# change r's value to 7 and set its error to 4, and set its range to 0-10
# CODE HERE

rs.test_1b(r)

r.Print() # see the impact of your changes

### Collections of variables: [RooArgSet](https://root.cern.ch/doc/v624/classRooArgSet.html) and [RooArgList](https://root.cern.ch/doc/v624/classRooArgList.html)
RooFit has two container types for collections of variables:

  1. A `RooArgList` behaves a bit like a python list (or c++ vector) of variables, i.e. access ith variable with `l[i]`
  2. A `RooArgSet`, is an unordered set of variables, but you can access variable "v" with `s["v"]`, so in this sense it is a bit like a python dictionary. 
  
Both types of collection are iterable, i.e. you can do things like:

```
for v in mySet:
  if not v.isCategory(): # this is (one way) how to know the 'type' of the variable
      print("Variable name = ", v.GetName(), " value = ", v.getVal())
```
  
You will encounter both in this tutorial. You need to know how to access the variables in the collections and how to copy values from one collection to another. Both collection types share many methods, here's some to be aware of:

| Method name | Description |
|-------------|-------------|
|`coll.Print()` | print names of the variables in the collection|
|`coll.Print("v")`| print names and values of the variables|
|`coll.size()`| number of variables in the collection (len(coll) also works)|
|`coll.find(vName)`| find variable by name, returns null object (`==None`) if not found|
|`coll.clone("newName")`| create a copy of the collection, but containing references to the same variables |
|`coll.snapshot()`| create an *indepedent copy* of the contents of the collection (i.e. copies the content as well) |
|`coll.assignValueOnly(another_coll)`| copy values from any variable in another_coll with matching names over to coll|
|`coll.selectCommon(another_coll)` | a new collection containing variables in coll that are also in another_coll (by name)|
|`coll.selectByAttrib("Constant",True)`| select only the constant variables (change to False to select floating)|
|`coll.setAttribAll("Constant",True)`| set all constant (change to False to make all floating)|
|`coll.add(var_or_coll`)| adding a variable to a collection or all variables in a collection to this collection|
|`coll.remove(var_or_coll)`| remove a variable or collection of variables from this collections |

Note that the `remove` method will only remove the variable that exactly matches the variable(s) passed to the method, i.e. it wont remove copies of the variables that have the same name but are independent (and therefore different objects). However, the `selectCommon` method will match variables by name, so you can remove by name with: `coll.remove(coll.selectCommon(other_coll))`


In [ ]:
s = rs.getObject("mySet")
l = rs.getObject("myList")

In [ ]:
# What is the name of last variable in l ? 
rs.test_2a( ... )

In [ ]:
# what is the value of variable called "bar" in s ?
rs.test_2b( ... )

In [ ]:
# Create a collection of the floating variables in s
rs.test_2c( ...  )

In [ ]:
# Create a collection of the variables that are in s but not in l (by name)
# hint: one solution would involve starting by creating a clone of the set
...
rs.test_2d( ... )

### Datasets: [RooDataSet](https://root.cern.ch/doc/v624/classRooDataSet.html)

Recall that a dataset is:

  * a table where the rows are (possibly weighted) "entries" and the columns are the observables, 
  * and an additional collection of global observable values. 
  
This information is held in RooFit's `RooDataSet` class. When a dataset is unbinned, the entries in the dataset are unweighted (i.e. each entry has a weight of 1) and so there is one entry for each event. When a dataset is binned, there is one entry for each bin of the model and the weight of the entry is equal to the number of events seen in that bin.

In this tutorial you wont be creating any datasets yourself so we will focus only on how you access the contents of a `RooDataSet`. Here are the most basic operations you might do with a dataset:

| Method name | Description |
|-------------|-------------|
|`d.get(i)`| Returns a `RooArgSet` of the ith entry (row) of a dataset |
|`d.numEntries()`| Number of entries in the dataset  |
|`d.weight()`| Weight of the row of the dataset that was last accessed with `get` |
|`d.sumEntries()`| Returns sum of the weights of all the entries in the dataset |
|`d.getGlobalObservables()`| If dataset has global observables, this returns a `RooArgSet` of them (returns null otherwise) |
|`d.Print()` | Print summary of number of entries in the dataset and the names of the observables |


In [ ]:
d = rs.getObject("myData")
d.Print()

In [ ]:
# what is the name of the discrete variable in the dataset?

...

rs.test_3a( varName )

In [ ]:
# what is the weight of the last entry in the dataset?

...

rs.test_3b( weight )

In [ ]:
# how many global observables are in this dataset?
rs.test_3c( ... )

### PDFs: [RooAbsPdf](https://root.cern.ch/doc/v624/classRooAbsPdf.html)
In RooFit a RooAbsPdf is just a **function of variables** with some complicated built-in machinary for normalizing the pdf. One of the early design choices with RooFit was that its PDFs shouldn't decide which of its variables are parameters and which are observables, it is basically up to the user to decide on any given evaluation of the PDF which should be treated as the observables. While this means the same PDF object can be used in different ways, it can be a bit counterintuitive to beginners. For now I think you should just accept that to evaluate a PDF you have to give it a list of the variables to treat as observables. Here's a small table of some of the methods of a pdf that are worth knowing about:

| Method name | Description |
|-------------|-------------|
|`p.getVariables()`| Returns a `RooArgSet` of all the pdfs variables (parameters and observables) |
|`p.getParameters(dataset)`| Just the variables that aren't part of the dataset, i.e. the parameters |
|`p.getObservables(dataset)`| The variables that are part of the dataset, i.e. the observables |
|`p.Print()` | Print some basic info about the pdf |
|`p.getVal(obs)` | Evaluate the pdf at the current variable values, treating the `RooArgSet` obs as observables |
|`p.servers()` | a list of the pdfs/functions/variables that this pdf directly depends on (see below) |
|`p.clients()` | a list of pdfs/functions that directly depend on this pdf. If this list is empty we have a 'top-level' pdf |

Get the variables of the function using `model.getVariables()` which returns a `RooArgSet` of all the variables of the model.

To evaluate the PDF at a particular point in the observable space, call `getVal(obs)` where obs is a RooArgSet of the variables of the PDF to treat as observables. you must first have set all the observables to the value you want to evaluate at. Note that you can create a `RooArgSet` containing a single variable using the constructor: `ROOT.RooArgSet(var)`

In [ ]:
m = rs.getObject("myModel")
m.Print()

In [ ]:
# Exercise: plot a graph of the PDF as a function of variable "x" (i.e. consider "x" to be the observable)
# Note: the PDF has other variables, these are considered the parameters -- leave them at their current values
# The plot should cover points from the min up to the max value of the variable range in steps of 0.2

graph = ROOT.TGraph()

var = ...

import numpy as np
for x in np.arange(... , ... ,0.2):
    ...
    graph.AddPoint(... , ...)

rs.test_4a(graph)

graph.Draw("AL")
ROOT.gPad.Draw() # show the current canvas

`RooAbsPdf`, like all functions in `RooFit`, are actually built as functions of other functions. In this way, all functions can be thought as trees (or graphs to use the proper terminology), where the nodes of the tree are the functions and the branches connect to the functions that the function depends on (the *servers*). The fundamental types (`RooRealVar` and `RooCategory`) are functions that depend on no other functions, so are the *leafs* of these trees. 

![Example of a tree graph](treegraph.png "Tree Graph")

In the above picture, nodes 8-14 would be fundamental types, and the other nodes would be functions or PDFs. 

In `RooFit` terminology, every function has a list of `servers` that it depends on, as well as a list of `clients` that depend on it:

  * A *top-level* pdf (or function) is one that has no clients (it's at the top of the tree).
  * The fundamental types have no servers (it's at the bottom of the tree).
  
The `clients()` and `servers()` method respectively return an iterable over the clients and servers of the function respectively. For example, you can use `len(pdf.clients())==0` to identify the top-level pdf in a list of pdfs.

### Saving to models and datasets: [RooWorkspace](https://root.cern.ch/doc/v624/classRooWorkspace.html)

The `RooWorkspace` is an object that can contains datasets, models, and in fact can store all sorts of other types of object. It is the standard way that `RooFit` objects are saved to a file. 

Note: The methods below work because a `RooArgSet` (or `RooArgList`) are allowed to contain functions and pdfs as well as fundamental type variables.

| Method name | Description |
|-------------|-------------|
|`w.allVars()`| Returns a `RooArgSet` of all the continuous-valued variables in the workspace. |
|`w.allCats()`| Returns a `RooArgSet` of all the discrete-valued variables in the workspace. |
|`w.allPdfs()`| Returns a `RooArgSet` of all the pdfs in the workspace. |
|`w.allData()`| Returns an `std::list` of the datasets in the workspace (can iterate over like a python list). |
|`w.getSnapshot(name)` | Returns a `RooArgSet` that has been saved to the workspace. The `name` is the name of the snapshot. |


In [ ]:
w = rs.getObject("myWorkspace")

In [ ]:
# What is the name of the top-level pdf in the workspace (hint: see the info about servers and clients of PDFs in the section above)

pdfName = ...

rs.test_5a( pdfName )

In [ ]:
# What is the name of the only dataset in the workspace?
# hint: convert std::list to python list with: py_list = [a for a in std_list]

dsName = ...

rs.test_5b( dsName )

In [ ]:
# What is the name and value of the global observable in the dataset

globName = ...
globVal = ...

rs.test_5c( globName, globVal )

### Fit Results: [RooFitResult](https://root.cern.ch/doc/v624/classRooFitResult.html)
A `RooFitResult` is the type of object used to hold the information about the result of a fit, which is just the minimization of a NLL function. Specifically, a `RooFitResult` has the following important methods:

| Method name | Description |
|-------------|-------------|
|`fr.status()` | Status code of the fit (an integer). 0 means the fit was good, non zero means some aspect of the result is suspect/untrustworthy. The interpretation of non-zero status codes is regretfully 'minimizer specific' i.e. it depends exactly how the fit was performed, but as a rule you should only fully trust fits with status = 0. |
|`fr.minNll()` | The post-fit value of the NLL function i.e. $-\text{ln}L(\alpha,\hat{\beta})$ where $\alpha$ are the constant parameters and $\beta$ are the floating parameters. |
|`ft.edm()` | The estimated 'vertical' distance to the minimum, which in many ways you can think of as the uncertainty on the `minNll` value |
|`fr.floatParsFinal()`| A `RooArgList` of the post-fit parameter values, with errors set to the hessian-estimated error. |
|`fr.constPars()` | A `RooArgList` of the parameter values of the constant parameters. |
|`fr.floatParsInit()` | A `RooArgList` of the pre-fit parameter values. |
|`fr.covarianceMatrix()` | Access the post-fit covariance matrix (as a `TMatrixD`) of the floating parameters. The order of the parameters in the `floatParsFinal()` list corresponds to the order of the elements in the covariance matrix. |

We haven't got any exercises for you to play with fit results ... but we will do a lot of that in the main session, so see you then!!!

### New for 2023: An Introduction to [xRooNode](https://gitlab.cern.ch/will/xroofit)

Partly based on feedback from these annual tutorials, a new high-level API is being developed for RooFit called xRooFit. Think of this xRooFit API being related to RooFit like how Keras is related to Tensorflow: you can still access the underlying RooFit objects but the xRooFit classes are there to assist and augment the user experience. This is still an experimental feature in ROOT (if you have your own version of ROOT you need the experimental root features turned on ... i.e. if you do `root-config --features` you need to see `root7` in the list), but we can have a quick play with it now....

The main class of this new API is called `xRooNode`. If you create a node from a workspace, you can then inspect/explore the content of the workspace using `Print()` and navigate through using `[]` operator. For example:

In [ ]:
ws = rs.getObject("myWorkspace")
w = ROOT.RooFit.Detail.XRooFit.xRooNode(ws) # creates an xRooNode out of the workspace
# we can print node to see the content ... use `depth=X` option to control how deep to print .. try changing X ...
w.Print("depth=2")

The above shows us this workspace consists of a dataset called `obsData` that has observables in it called `channelCat,m_obs,globs_alpha_par`. There's also a model called `simPdf` which has two children under it (these are what we call "channels" of a model) and these channels appear to be made up of samples and a gaussian constraint term. 

We can access the parameters and observables of the model like this:

In [ ]:
w["simPdf"].pars().Print() # change "pars" to "obs" to get observables instead
w["simPdf"].pars()["mu"].setVal(10) # example of accessing a parameter and changing its value

We can even visualize the model (including the dataset) very quickly like this:

In [ ]:
w["simPdf"].Draw("e auxRatio") # draws model with error bars and a ratio plot
ROOT.gPad.Draw()

Or we can access individual bin contents like this:

In [ ]:
for channel in w["simPdf"]:
    print(channel.GetName(),":")
    for sample in channel["samples"]:
        print(" ",sample.GetName(),":")
        for b in sample.bins():
            print(" "," ",b.GetContent())